# Importações

In [1]:
from Carga import Carga
from pandas import pandas as pd
import numpy as np
import unidecode
import Database
import json


In [2]:
#Re-importanto a biblioteca porque foi modificada - pode ser ignorado
#import importlib
#importlib.reload(Database)

In [3]:
db = Database.Database('CNPq')

## Carregando tabela de indicadores

In [1]:
engine = Carga.db_engine()
dt = pd.read_sql("indicadores_pareamento", engine)


NameError: name 'Carga' is not defined

## Pegando lista de Ids já realizados

In [5]:
sql = 'SELECT DISTINCT id FROM pareamentos;'
ids_já_feitos = db.query(sql)

In [6]:
ids_já_feitos = [num[0] for num in ids_já_feitos]

In [ ]:
len (ids_já_feitos)

## Pegando lista de Ids para fazer

In [8]:
sql = 'select distinct id from demanda_bruta inner join pagamentos on demanda_bruta."Processo" = pagamentos."Processo"'

In [9]:
ids_para_fazer = db.query(sql)

In [10]:
asd = []
for num in ids_para_fazer:
    if int(num[0]) not in ids_já_feitos:
        asd.append(int(num[0]))
ids_para_fazer = asd
del asd

In [11]:
len(ids_para_fazer)

2887

## Retirada dos não doutores - Existem 3 não doutores contemplados!!!

In [12]:
dt = dt[pd.notnull(dt.ano_doutorado)]

## Normalizando as Tabelas

In [13]:
dt.qty_2012 = dt.qty_2012.fillna(0)
dt.qty_2013 = dt.qty_2013.fillna(0)
dt.qty_2014 = dt.qty_2014.fillna(0)
dt.qty_2015 = dt.qty_2015.fillna(0)
dt.qty_2016 = dt.qty_2016.fillna(0)
dt.qty_2017 = dt.qty_2017.fillna(0)
dt.qty_2018 = dt.qty_2018.fillna(0)
dt.qty_2019 = dt.qty_2019.fillna(0)

In [14]:
dt.area_demanda_bruta = dt.area_demanda_bruta.fillna('nenhum')
dt.areas_conhecimento = dt.areas_conhecimento.fillna('nenhum')
dt.tipos_vinculo = dt.tipos_vinculo.fillna('nenhum')
dt.enquadramento_vinculo = dt.enquadramento_vinculo.fillna('nenhum')

In [15]:
dt.area_demanda_bruta = dt.area_demanda_bruta.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()

In [16]:
dt.areas_conhecimento = dt.areas_conhecimento.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()

In [17]:
dt.tipos_vinculo = dt.tipos_vinculo.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()

In [18]:
dt.enquadramento_vinculo = dt.enquadramento_vinculo.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()

## Criando tabelas parciais

In [19]:
financiados = dt[dt.pgtos > 0]

In [22]:
dt = dt[pd.isnull(dt.pgtos)]

## Verificando número de pareamentos por ID - Pareando

In [ ]:
min_para_parear = 3
num_pareamentos = 3
erros_ids_inexistentes_pareamento = []
for id in ids_para_fazer:
    #Pegando dados do ID
    print('Fazendo id: ', id)
    financiado = financiados[financiados.id == id]
    if financiado.empty:
        erros_ids_inexistentes_pareamento.append(id)
        with open('d:\erros_ids_inexistentes_pareamento.json', 'w') as f:
            json.dump(erros_ids_inexistentes_pareamento, f)
        continue      
    
    #Pegando áreas do Conhecimento
    financiado.areas_conhecimento.iloc[0].split(',')
    
    #Pegando Regiões
    região = financiado.iloc[0].uf
    tipo_região = None
    if not região == None:
        if região in ('SP'):
            tipo_região = 1
        elif região in ('MG', 'RS', 'RJ', 'PR'):
            tipo_região = 2
        else:
            tipo_região = 3
    
    #Pegando ano do Indicador a ser usado
    if financiado.chamada.str.contains('2012', na=False).unique()[0] == True : ano = 2012
    elif financiado.chamada.str.contains('2013', na=False).unique()[0] == True : ano = 2013
    elif financiado.chamada.str.contains('2014', na=False).unique()[0] == True : ano = 2014
    else: ano = None

    #Pegando Faixa do Financiado
    if financiado.chamada.str.contains('Faixa A', na=False).unique()[0] == True : faixa = 'A'
    elif financiado.chamada.str.contains('Faixa B', na=False).unique()[0] == True : faixa = 'B'
    elif financiado.chamada.str.contains('Faixa c', na=False).unique()[0] == True : faixa = 'C'
    else: faixa = None
        
    #Pegando Enquadramentos Funcionais Atuais
    enquadramento_vinculo = financiado.enquadramento_vinculo.unique()
    if not enquadramento_vinculo == None and len(enquadramento_vinculo) > 0:
        if enquadramento_vinculo[0].find('professor') > -1:
            vinculo = 'professor'
        else:
            vinculo = 'outro'

    tipos_vinculo = financiado.enquadramento_vinculo.unique()
    if not tipos_vinculo == None and len(tipos_vinculo) > 0:   
        if tipos_vinculo[0].find('servidor_publico') > -1:
            enquadramento = 'servidor_publico'
        elif tipos_vinculo[0].find('celetista') > -1:
            enquadramento = 'celetista'
        elif tipos_vinculo[0].find('professor_visitante') > -1:
            enquadramento = 'professor_visitante' 
        elif tipos_vinculo[0].find('colaborador') > -1:
            enquadramento = 'colaborador' 
        elif tipos_vinculo[0].find('bolsista') > -1:
            enquadramento = 'bolsista' 
        elif tipos_vinculo[0].find('livre') > -1:
            enquadramento = 'livre' 
        elif tipos_vinculo[0].find('outro') > -1:
            enquadramento = 'outro'     
    else: enquadramento = 'nenhum'
        
    #Fazendo o Pareamento
    
    pareados = dt.loc[(dt.areas_conhecimento.str.contains(financiado.area_demanda_bruta.unique()[0].split('(')[0].strip(), na=False)]
    if pareados.size > min_para_parear:
        if vinculo = 'professor':
            pareados2 = pareados.loc[(dt.enquadramento_vinculo.str.contains('professor', na=False))]
        else:
            pareados2 = pareados.loc[~(dt.enquadramento_vinculo.str.contains('professor', na=False))]
    if pareados.size > min_para_parear:
        pareados = pareados2           
        pareados2 = pareados.loc[(dt.tipos_vinculo.str.contains(enquadramento, na=False))]
    if pareados2.size > min_para_parear:
        pareados = pareados2
        pareados2 = pareados.loc[
            (dt.ano_doutorado < financiado.ano_doutorado.unique()[0] + 10) &
            (dt.ano_doutorado > financiado.ano_doutorado.unique()[0] - 10)
        ]
    if pareados2.size > min_para_parear:
        pareados = pareados2
        pareados2 = pareados.loc[
            (dt.ano_doutorado < financiado.ano_doutorado.unique()[0] + 5) &
            (dt.ano_doutorado > financiado.ano_doutorado.unique()[0] - 5)
        ]
    if pareados2.size > min_para_parear:
        pareados = pareados2
        pareados2 = pareados.loc[(dt.sexo == financiado.sexo.unique()[0])]
    if pareados2.size > min_para_parear:
        pareados = pareados2
        if tipo_região == 1:
            pareados2 = pareados.loc[pareados.uf.isin(('SP'))]
        elif tipo_região == 2:
            pareados2 = pareados.loc[pareados.uf.isin(('MG', 'RS', 'RJ', 'PR'))]
        elif tipo_região == 3:
            pareados2 = pareados.loc[~pareados.uf.isin(('SP', 'MG', 'RS', 'RJ', 'PR'))]
    if pareados2.size > min_para_parear:
        pareados = pareados2
                       
    #Fezendo lista de erros quadráticos
    ids_pareados = []
    if not pareados.empty:
        for grupo in financiado.grupo.unique():
            ind_financiado = financiado.loc[(financiado.grupo == grupo)]['qty_' + str(ano)].iloc[0]
            #print('\n\n', grupo, ind_financiado, ': ')

            for id_pareado in pareados.id.unique():
                ind_pareado = pareados.loc[
                    (pareados.grupo == grupo)
                    & (pareados.id == id_pareado)
                    ]['qty_' + str(ano)]
                if len(ind_pareado) > 0:
                    ind_pareado = ind_pareado.iloc[0]
                else:
                    ind_pareado = 0
                erro = {
                    'id': id,
                    'id_pareado': id_pareado,
                    'grupo': grupo,
                    'valor': ind_pareado,
                    'erro2': (ind_financiado - ind_pareado)**2
                }
                ids_pareados.append(erro)

        #Acrescentando Anos de Doutor como um Indicador
        for id_pareado in pareados.id.unique():
            ind_pareado = pareados.ano_doutorado.loc[
                (pareados.id == id_pareado)
                ].unique()
            #print(ind_pareado, type(ind_pareado))
            if len(ind_pareado) > 0:
                ind_pareado = ind_pareado[0]
            else:
                ind_pareado = 0
            erro = {
                'id': id,
                'id_pareado': id_pareado,
                'grupo': "Anos de Doutor",
                'valor': ind_pareado,
                'erro2': (financiado.ano_doutorado.iloc[0] - ind_pareado)**2
            }
            ids_pareados.append(erro)

        #Acrescentando Áreas do Conhecimento em Comum como outro indicador
        areas_conhecimento_financiado = financiado.areas_conhecimento.iloc[0]
        if not areas_conhecimento_financiado == None:
            areas_conhecimento_financiado = areas_conhecimento_financiado.split(',')
            total_areas = len(areas_conhecimento_financiado)

            for id_pareado in pareados.id.unique():
                areas_em_comum = 0
                lista_areas_conhecimento = pareados.loc[pareados.id == id_pareado].areas_conhecimento
                if len(lista_areas_conhecimento) > 0 and not lista_areas_conhecimento.iloc[0] == None:
                    lista_areas_conhecimento = lista_areas_conhecimento.iloc[0].split(',')
                    for area in areas_conhecimento_financiado:
                        if area in lista_areas_conhecimento:
                            areas_em_comum += 1
                erro = {
                    'id': id,
                    'id_pareado': id_pareado,
                    'grupo': 'areas_conhecimento',
                    'valor': areas_em_comum,
                    'erro2': (total_areas - areas_em_comum)**2
                }
                ids_pareados.append(erro)    
            
    #Calculando o Erro
    erro = pd.DataFrame(ids_pareados)
    #print(erro)
    desvio_padrão = []
    if not erro.empty:
        for id_pareado in erro.id_pareado.unique():
            erro_id = erro.loc[erro.id_pareado == id_pareado]
            if not erro_id.empty:
                soma_erro = erro_id.erro2.sum()
                num_erros = len(erro_id.index)
                err = {
                    'id': id,
                    'id_pareado': id_pareado,
                    'erro': np.sqrt(soma_erro/num_erros)
                }
                desvio_padrão.append(err)

                db.insert_dict("pareamentos", err, on_conflict=['id', 'id_pareado'])
            

In [ ]:
db.commit()

### Erro para parar o código

In [ ]:
efghrertherthhter
print estou na fazendo sem parênteses ou aspas

In [ ]:
pareados

In [ ]:
pareados = dt.loc[(dt.areas_conhecimento.str.contains(financiado.area_demanda_bruta.unique()[0].split('(')[0].strip(), na=False))]

In [ ]:
pareados

In [ ]:
financiado.area_demanda_bruta.unique()[0].split('(')[0]